# Restructuring Data in Pandas

This notebook is inspired from the textbook:

[Jake VanderPlas: Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/index.html)

In [3]:
import pandas as pd
import numpy as np

## Agenda

- How to read data with different encodings
- How to find missing values in dataframes
- Dealing with missing values
- Combining dataframes
- Groupby method in Pandas
- Aggregation with groupby

## Reading Data - Encodings

[Download Data](https://www.kaggle.com/kwullum/fatal-police-shootings-in-the-us?select=PoliceKillingsUS.csv)

[Washington Post Repo](https://github.com/washingtonpost/data-police-shootings)

In [48]:
df = pd.read_csv('data/datasets_2647_4395_PoliceKillingsUS.csv', encoding= 'ascii')

UnicodeDecodeError: 'ascii' codec can't decode byte 0x96 in position 25272: ordinal not in range(128)

### Encodings

In [38]:
before = "This is the euro symbol: €"

In [39]:
after = before.encode()

after

b'This is the euro symbol: \xe2\x82\xac'

In [42]:
after.decode('ascii')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 25: ordinal not in range(128)

In [43]:
before.encode(encoding = 'ascii', errors = 'replace')

b'This is the euro symbol: ?'

In [17]:
## try decoding with utf-8
## try decoding with ascii

## encode with ascii make sure that setting errors = 'replace'
## decoce with using ascii

### Solution to the encoding-decoding mess chardet library

[Chardet package](https://pypi.org/project/chardet/)

In [13]:
import sys
!{sys.executable} -m pip install chardet

In [45]:
import chardet

[For more on installing packages from notebook](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)

In [49]:
import chardet
# look at the first ten thousand bytes to guess the character encoding
with open("data/datasets_2647_4395_PoliceKillingsUS.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(50000))

# check what the character encoding might be
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [18]:
## try encoding = 'ascii'
## read the error carefully did it changed?

- [Kaggle - Dealing with Char Encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings)

- [Python Standard Encodings](https://docs.python.org/3/library/codecs.html#standard-encodings)

In [50]:
df = pd.read_csv('data/datasets_2647_4395_PoliceKillingsUS.csv', encoding='Windows-1252')

In [51]:
df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


In [52]:
print(df.shape)
df.head();

(2535, 14)


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2535 non-null   int64  
 1   name                     2535 non-null   object 
 2   date                     2535 non-null   object 
 3   manner_of_death          2535 non-null   object 
 4   armed                    2526 non-null   object 
 5   age                      2458 non-null   float64
 6   gender                   2535 non-null   object 
 7   race                     2340 non-null   object 
 8   city                     2535 non-null   object 
 9   state                    2535 non-null   object 
 10  signs_of_mental_illness  2535 non-null   bool   
 11  threat_level             2535 non-null   object 
 12  flee                     2470 non-null   object 
 13  body_camera              2535 non-null   bool   
dtypes: bool(2), float64(1), 

### Detecting Null Values

In [25]:
## find null values in each column

In [60]:
df.isna().sum()

id                           0
name                         0
date                         0
manner_of_death              0
armed                        9
age                         77
gender                       0
race                       195
city                         0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                        65
body_camera                  0
dtype: int64

[Difference between `isna` and  `isnull`](https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas)

## Dealing with Missing Values

### Droping Rows with NaN values

In [26]:
## use of dropna with default

In [58]:
df.dropna()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,2805,Aries Clark,25/07/17,shot,undetermined,16.0,M,B,Marion,AR,False,undetermined,Not fleeing,False
2523,2808,Kesharn K. Burney,26/07/17,shot,vehicle,25.0,M,B,Dayton,OH,False,attack,Car,False
2525,2820,Deltra Henderson,27/07/17,shot,gun,39.0,M,B,Homer,LA,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


In [61]:
## Drop rows with more than 2 missing values only

In [62]:
df.dropna(thresh= 12)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,2822,Rodney E. Jacobs,28/07/17,shot,gun,31.0,M,NaN,Kansas City,MO,False,attack,Not fleeing,False
2531,2813,TK TK,28/07/17,shot,vehicle,NaN,M,NaN,Albuquerque,NM,False,attack,Car,False
2532,2818,Dennis W. Robinson,29/07/17,shot,gun,48.0,M,NaN,Melba,ID,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


In [64]:
df.isna().sum()

id                           0
name                         0
date                         0
manner_of_death              0
armed                        9
age                         77
gender                       0
race                       195
city                         0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                        65
body_camera                  0
dtype: int64

In [29]:
## how to drop only rows?

## how to drop columns?

## how about the other parameters: how, thresh, inplace, subset

In [66]:
df.dropna(subset = ['age', 'race'], how = 'all')

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,2819,Brian J. Skinner,28/07/17,shot,knife,32.0,M,NaN,Glenville,NY,True,other,Not fleeing,False
2530,2822,Rodney E. Jacobs,28/07/17,shot,gun,31.0,M,NaN,Kansas City,MO,False,attack,Not fleeing,False
2532,2818,Dennis W. Robinson,29/07/17,shot,gun,48.0,M,NaN,Melba,ID,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


### Filling Null Values

In [30]:
## droping might not be the best practice


In [67]:
## replace missing values in age column with mean age value:

In [70]:
mean_age = int(df.age.mean())
mean_age

36

In [76]:
df['age'].fillna(value= mean_age,inplace = True )

In [77]:
df.age.isna().sum()

0

In [83]:
df.race.value_counts()

W    1201
B     618
H     423
A      39
N      31
O      28
Name: race, dtype: int64

In [85]:
df.city.value_counts()

Los Angeles     39
Phoenix         31
Houston         27
Chicago         25
Las Vegas       21
                ..
Campo            1
Claremont        1
Elkhart          1
Hometown         1
City Terrace     1
Name: city, Length: 1417, dtype: int64

[Pandas Fillna - Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

## `Apply`, `Map` and `ApplyMap`

[Dataset](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-cities-and-towns.html)

In [31]:
## reading data from excel sheet

In [100]:
census = pd.read_excel('data/SUB-IP-EST2019-ANNRNK.xlsx', skiprows = 3, skipfooter= 5)

In [101]:
census.head(20)

,Unnamed: 0,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817
1,2,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576
2,3,"Chicago city, Illinois",2695598,2695652,2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976
3,4,"Houston city, Texas",2099451,2095517,2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268
4,5,"Phoenix city, Arizona",1445632,1446691,1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992
5,6,"Philadelphia city, Pennsylvania",1526006,1526012,1528283,1540466,1551824,1558313,1565460,1571065,1576051,1580601,1583592,1584064
6,7,"San Antonio city, Texas",1327407,1326161,1332299,1357120,1383075,1408339,1435456,1464043,1487843,1511154,1530016,1547253
7,8,"San Diego city, California",1307402,1301929,1305906,1319592,1336776,1355320,1375831,1387323,1402089,1412621,1421917,1423851
8,9,"Dallas city, Texas",1197816,1197658,1200350,1218282,1242115,1258835,1279098,1301329,1323916,1342479,1341802,1343573
9,10,"San Jose city, California",945942,952528,954940,970369,983530,1001279,1014273,1025980,1030242,1032335,1028020,1021795


In [102]:
## use default? Is it ok?

## use skiprows. Are we done?

In [103]:
## dropping columns

## renaming columns

In [104]:
census.drop(columns= ['Unnamed: 0'], inplace = True, axis = 'columns')

In [105]:
census.head(2)

,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576


In [106]:
## rename a column

In [112]:
census.columns.to_list()

['Unnamed: 1',
 'Census',
 'Estimates Base',
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [113]:
census.rename(columns= {'Unnamed: 1': 'Place', 2010: 'Murat'})

,Place,Census,Estimates Base,Murat,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576
2,"Chicago city, Illinois",2695598,2695652,2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976
3,"Houston city, Texas",2099451,2095517,2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268
4,"Phoenix city, Arizona",1445632,1446691,1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992
...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,"Lakewood city, Ohio",52131,52131,52026,51662,51400,51257,51088,50811,50528,50211,50012,49678
784,"Troy city, New York",50129,50162,50081,50020,49848,49921,49968,49776,49597,49433,49354,49154
785,"Saginaw city, Michigan",51508,51469,51363,51012,50728,50272,49798,49230,48902,48696,48310,48115
786,"Niagara Falls city, New York",50193,50031,49996,49760,49526,49328,49083,48769,48526,48249,48012,47720


### Apply Method

In [116]:
### Apply method work with dataframes and not with series

## The function you provide in Apply takes columns or rows in it.

In [120]:
census[[2010,2011, 2012, 2013, 2014, 2015]].apply(lambda x: x.std(),)

2010    371186.233807
2011    374827.872540
2012    378482.166172
2013    381345.358326
2014    383889.441597
2015    386186.938528
dtype: float64

### Applymap Method

In [35]:
## try applymap as above

In [127]:
census[[2010,2011, 2012, 2013, 2014, 2015]].applymap(lambda x: np.max(x))

,2010,2011,2012,2013,2014,2015
0,8190209,8272948,8346693,8396091,8433806,8463049
1,3795512,3820876,3851202,3881622,3909901,3938568
2,2697477,2708114,2719141,2725731,2727066,2724344
3,2100280,2126032,2161593,2199391,2241826,2286908
4,1449038,1469796,1499274,1526491,1555445,1583690
...,...,...,...,...,...,...
783,52026,51662,51400,51257,51088,50811
784,50081,50020,49848,49921,49968,49776
785,51363,51012,50728,50272,49798,49230
786,49996,49760,49526,49328,49083,48769


In [ ]:
## Splitting cities and states 

In [129]:
census.head()

,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576
2,"Chicago city, Illinois",2695598,2695652,2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976
3,"Houston city, Texas",2099451,2095517,2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268
4,"Phoenix city, Arizona",1445632,1446691,1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992


In [140]:
census['city'] = census['Unnamed: 1'].map(lambda x: x.split(',')[0][:-5])

In [143]:
census['state']= census['Unnamed: 1'].map(lambda x: x.split(',')[1].strip(' '))

In [146]:
census.state[453]

'California'

In [148]:
census.head()

,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,city,state
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817,New York,New York
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576,Los Angeles,California
2,"Chicago city, Illinois",2695598,2695652,2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976,Chicago,Illinois
3,"Houston city, Texas",2099451,2095517,2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268,Houston,Texas
4,"Phoenix city, Arizona",1445632,1446691,1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992,Phoenix,Arizona


In [147]:
df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


In [128]:
## Map method: This works with series objects. 

In [ ]:
## creating the abbrevated state names

In [149]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [150]:
states = {values: key for key, values in states.items()}

In [151]:
states

{'Alaska': 'AK',
 'Alabama': 'AL',
 'Arkansas': 'AR',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'District of Columbia': 'DC',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Iowa': 'IA',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Massachusetts': 'MA',
 'Maryland': 'MD',
 'Maine': 'ME',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Missouri': 'MO',
 'Northern Mariana Islands': 'MP',
 'Mississippi': 'MS',
 'Montana': 'MT',
 'National': 'NA',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Nebraska': 'NE',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'Nevada': 'NV',
 'New York': 'NY',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 

In [153]:
census['abbrevated'] = census.state.map(lambda s: states[s])

In [154]:
census.head()

,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,city,state,abbrevated
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817,New York,New York,NY
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576,Los Angeles,California,CA
2,"Chicago city, Illinois",2695598,2695652,2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976,Chicago,Illinois,IL
3,"Houston city, Texas",2099451,2095517,2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268,Houston,Texas,TX
4,"Phoenix city, Arizona",1445632,1446691,1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992,Phoenix,Arizona,AZ


In [ ]:
states = {values: key for key, values in states.items()}

In [ ]:
census['abbr'] = census.state.map(lambda state: states[state])

In [ ]:
census.head(2)

[More on the differences between `apply`,`map` and `applymap`](https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas)

## Merging Datasets

This Part of the notebook is adopted from:

[PythonDataScienceHandbook - Merge -Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

In [156]:
import pandas as pd
import numpy as np


## this is only for the visualization purposes 

## you don't have to understand this right now
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [157]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [158]:


df3 = pd.merge(df1, df2)
display('df1', 'df2','pd.merge(df1, df2)')



df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2)
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

In [159]:


df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')



df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

In [160]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


In [163]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

For more details please check this excellent notebook:

[Python Data Science Handbook - Merge-Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

And the `pandas` documentation

[Pandas - Merging](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [164]:
census.head(2)

,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,city,state,abbrevated
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817,New York,New York,NY
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576,Los Angeles,California,CA


In [165]:
df.head(2)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False


In [175]:
df_merged = pd.merge(census[['city', 'state', 2016]], df, on = 'city', how = 'inner', suffixes= ['_census', '_police'])

In [176]:
df_merged.head()

,city,state_census,2016,id,name,date,manner_of_death,armed,age,gender,race,state_police,signs_of_mental_illness,threat_level,flee,body_camera
0,New York,New York,8469153,411,David Felix,25/04/15,shot,unarmed,24.0,M,B,NY,True,attack,Foot,False
1,New York,New York,8469153,1470,George Tillman,17/04/16,shot,gun,32.0,M,B,NY,False,attack,Foot,False
2,New York,New York,8469153,1543,Garry Conrad,18/05/16,shot,knife,46.0,M,W,NY,True,other,Not fleeing,False
3,New York,New York,8469153,1631,Rashaun Lloyd,19/06/16,shot,gun,25.0,M,B,NY,False,attack,Not fleeing,False
4,New York,New York,8469153,1706,Orville Edwards,11/07/16,shot,gun,39.0,M,B,NY,False,attack,Foot,False


## Groupby

In [ ]:
grouped = df_merged.groupby(by = 'city')
grouped.get_group('New York').race.value_counts()

In [ ]:
grouped[2016].mean()

In [ ]:
grouped.aggregate(['min', np.median, max])['age']

In [ ]:
my_func = lambda x: x.max() - x.min()

In [ ]:
grouped.aggregate({'date': 'max', 'age': 'min', 2016: my_func})

In [ ]:
grouped = df_merged.groupby(by = ['city', 'state_x'])


In [ ]:
grouped.get_group(('New York', 'New York')).race.value_counts()

In [ ]:
grouped[2016].mean()

In [ ]:
(grouped['city'].count()/grouped[2016].mean()).sort_values(ascending =False)

In [ ]:
grouped.get_group('Toledo')

## Resources

[More on encodings](https://realpython.com/python-encodings-guide/)